In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
if torch.cuda.is_available(): 
 print("cuda") 
else: 
 print("cpu") 

In [9]:
import os
from PIL import Image
from IPython.display import display

dataset_dir = './data/Dataset'

def process_image(image_path):
    img = Image.open(image_path)
    return img

def create_data_logger(dataset_dir):
    data = []
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith('.png'):
                image_path = os.path.join(root, file)
                label = os.path.basename(file)[16]
                img = process_image(image_path)
                data.append({'image': img, 'label': label, 'path': image_path})
    return data

dataset = create_data_logger(dataset_dir)
print("img")
display(dataset[0]['image'])
print("label")
print(dataset[0]['label'])


OSError: [Errno 24] Too many open files: './data/Dataset\\s0001_00001_0_0_0_0_0_01.png'